# Assignment 3: Binary options, Barrier options, and Implied volatility on the Heston model

#### Job Marcelis, Ernani Hazbolatow, Koen Verlaan
#### May 19, 2025 University of Amsterdam

In [ ]:
from src.binary_PDE import *

#### Binary options

In [ ]:
# Main simulation settings
Nx = 100
Nt = 100
S0 = 1
r = 0.05
sigma = 0.2
K = 1.05
T = 1
num_paths = 10000
seed = 420
q = 2*r/sigma**2
a = 0.5*(q - 1)
b = 0.25*(q + 1)**2
xmax = 1
x = np.linspace(-xmax, xmax, Nx)
tau = np.linspace(0, 0.5*sigma**2 * T, Nt)

Below we compute the binary option price using the analytical formula, Monte Carlo simulations, the Backward-Time-Central-Space (BTCS) scheme, and the Crank-Nicolson (CN) scheme. This is done at $S_0 = 1$ and as a function of time. Since the $K > S_0$, the price approaches 0 as $T \rightarrow 1$. If $K < S_0$, the price would become 1.

In [ ]:
Nt = 40
tau = np.linspace(0, 0.5*sigma**2 * T, Nt)

######## Analytical Price and Monte Carlo price ##############
time = np.linspace(0, 0.999, 20)
analytical_prices = np.zeros(len(time))
MC_prices = np.zeros_like(analytical_prices)
MC_CIs = np.zeros_like(analytical_prices)
for i, T in enumerate(time):
    price = closed_form_price(1-T, r, sigma, K, S0)
    MC_price, MC_CI = MC_binary_price(num_paths, 1-T, r, sigma, K, S0, seed+i)

    analytical_prices[i] = price
    MC_prices[i] = MC_price
    MC_CIs[i] = MC_CI

########## BTCS price ############
result_BTCS = BTCS_scheme(K, x, tau, q, a, b)
x0 = np.log(S0 / K)
y_at_x0 = np.zeros(len(tau))
for n in range(len(tau)):
    f = interp1d(x, result_BTCS[n, :], kind='linear', fill_value='extrapolate')
    y_at_x0[n] = f(x0)
C_at_S0_BTCS = K * y_at_x0 * np.exp(-a*x0 - b*tau)
t_vals = T - 2*tau / sigma**2

############ CN price ##############
result_CN = CN_scheme(K, x, tau, q, a, b)
y_at_x0 = np.zeros(len(tau))
for n in range(len(tau)):
    f = interp1d(x, result_CN[n, :], kind='linear', fill_value='extrapolate')
    y_at_x0[n] = f(x0)
C_at_S0_CN = K * y_at_x0 * np.exp(-a*x0 - b*tau)


plt.figure(figsize=(7,5), dpi=300)
plt.scatter(t_vals, C_at_S0_BTCS, color='black', label='BTCS Scheme', marker='v')
plt.scatter(t_vals, C_at_S0_CN, color='green', label='CN Scheme', marker='s')
plt.scatter(time, analytical_prices, label='Analytical', color='red', marker='*', zorder=5)
plt.errorbar(time, MC_prices, yerr=MC_CIs, fmt='o', label='MC', color='blue')
plt.xlabel(r'$t$', fontsize=16)
plt.ylabel(r'$C(t, S=1)$', fontsize=16)
plt.title(r'Time evolution of Binary Call for $S_0=1$, $K=1.05$', fontsize=17)
plt.grid(ls='dashed')
plt.tick_params(axis='both', labelsize=13)
plt.legend(fontsize=13)
plt.tight_layout()
plt.show()

Below is the option price surface as a function of t and S.

In [ ]:
Nt = 100
tau = np.linspace(0, 0.5*sigma**2 * T, Nt)
result_CN = CN_scheme(K, x, tau, q, a, b)

plot_option_surface(result_CN, tau, x, T, K, sigma, a, b)

To explore the parameter space, we vary $K$ and $\sigma$ between reasonable values while keeping the rest constant, we then set $t=0$ and plot the option price vs. $K$ and $S$, and $\sigma$ and $S$, respectively.

In [ ]:
K_vals = np.linspace(0.8, 1.2, 40)
K_mesh, S_mesh, C_surface = sens_analysis(x, tau, Nx, r, sigma, K, K_vals, parameter_to_vary='K')

fig = plt.figure(figsize=(10, 7), dpi=300)
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(K_mesh, S_mesh, C_surface, cmap='viridis', edgecolor='none')
ax.set_xlabel(r'$K$', fontsize=16, labelpad=10)
ax.set_ylabel(r'$S$', fontsize=16, labelpad=10)
ax.set_zlabel(r'$C(t=0,S;K)$', fontsize=16, labelpad=10)
ax.set_title(r'Binary Call Price vs $S$ and $K$', fontsize=18, pad=10)
ax.tick_params(axis='both', labelsize=14)
cbar = fig.colorbar(surf, shrink=0.4, aspect=10, pad=0.05)
cbar.set_label('Binary Option Price', fontsize=14, labelpad=5)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

In [ ]:
sigma_vals = np.linspace(0.05, 0.5, 20)
sigma_mesh, S_mesh, C_surface = sens_analysis(x, tau, Nx, r, sigma, K, sigma_vals, parameter_to_vary='sigma')

fig = plt.figure(figsize=(10, 7), dpi=300)
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(sigma_mesh, S_mesh, C_surface, cmap='viridis', edgecolor='none')
ax.set_xlabel(r'$\sigma$', fontsize=16, labelpad=10)
ax.set_ylabel(r'$S$', fontsize=16, labelpad=10)
ax.set_zlabel(r'$C(t=0,S)$', fontsize=16, labelpad=10)
ax.set_title(r'Binary Call Price vs $\sigma$ and $S$', fontsize=18, pad=10)
ax.tick_params(axis='both', labelsize=14)
cbar = fig.colorbar(surf, shrink=0.4, aspect=10, pad=0.05)
cbar.set_label('Binary Option Price', fontsize=14, labelpad=5)
cbar.ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

Finally, we compute the option $\Delta$ using the analytical formula as well as the approximated $\Delta$ from the crank-nicolson results.

In [ ]:
S_grid = K * np.exp(x)
delta_a = digital_delta_analytic(r, sigma, S_grid, K, T)

C_grid = K * result_CN[-1, :] * np.exp(-a*x - b*tau[-1])
delta_CN = np.gradient(C_grid, S_grid)

plt.figure(figsize=(7, 5), dpi=300)
plt.title(r'Option $\Delta$ of a Binary Option', fontsize=17)
plt.plot(S_grid, delta_CN, label='CN Δ', color='blue')
plt.plot(S_grid, delta_a,  '--', label='Analytic Δ', color='red')
plt.legend(fontsize=12)
plt.xlabel(r'$S$', fontsize=15)
plt.ylabel(r'$\Delta$', fontsize=15)
plt.tight_layout()
plt.grid(ls='dashed')
plt.tick_params(axis='both', labelsize=13)
plt.show()